In [ ]:
from __future__ import annotations

from typing import Any

from example_models import get_tpi_ald_model
from modelbase2 import (
    LabelMapper,
    LinearLabelMapper,
    Simulator,
    plot,
)
from modelbase2.types import unwrap2


def print_annotated(description: str, value: Any) -> None:
    print(
        description,
        value,
        sep="\n",
        end="\n\n",
    )

## Labeled models

Labelled models allow explicitly mapping the transitions between isotopomers variables.  
This, for example, allows building models of the Calvin-Benson-Bassham cycle, in which each carbon atom can be labelled individually:


<img src="assets/cbb-labeled.png" style="max-width: 30rem;">


*modelbase* includes a `LabelMapper` that takes

- a model
- a dictionary mapping the variables to the amount of label positions they have
- a transition map 

to auto-generate all possible `2^n` variants of the variables and reaction transitions between them.  

As an example let's take triose phosphate isomerase, which catalyzes the interconversion of glyceraldehyde 3-phosphate (GAP) and dihydroxyacetone phosphate (DHAP).  
As illustrated below, in the case of the forward reaction the first and last carbon atoms are swapped

<img src="assets/carbon-maps.png" style="max-width: 250px">

So DHAP(1) is build from GAP(3), DHAP(2) from GAP(2) and DHAP(3) from GAP(1).  
We notate this using normal **0-based indexing** as follows

```python
label_maps = {"TPIf": [2, 1, 0]}
```

In [ ]:
mapper = LabelMapper(
    get_tpi_ald_model(),
    label_variables={"GAP": 3, "DHAP": 3, "FBP": 6},
    label_maps={
        "TPIf": [2, 1, 0],
        "TPIr": [2, 1, 0],
        "ALDf": [0, 1, 2, 3, 4, 5],
        "ALDr": [0, 1, 2, 3, 4, 5],
    },
)

if (
    concs := Simulator(mapper.build_model(initial_labels={"GAP": 0}))
    .simulate(20)
    .get_full_concs()
) is not None:
    plot.relative_label_distribution(mapper, concs, n_cols=3)

### Linear label mapper


The `LabelMapper` makes no assumptions about the state of the model, which causes a lot of complexity.  
In steady-state however, the space of possible solutions is reduced and the labelling dynamics can be represented by a set of linear differential equations.  
See [Sokol and Portais 2015](https://doi.org/10.1371/journal.pone.0144652) for the theory of dynamic label propagation under the stationary assumption.


In [ ]:
m = get_tpi_ald_model()

concs, fluxes = unwrap2(Simulator(m).simulate(20).get_concs_and_fluxes())


mapper = LinearLabelMapper(
    m,
    label_variables={"GAP": 3, "DHAP": 3, "FBP": 6},
    label_maps={
        "TPIf": [2, 1, 0],
        "TPIr": [2, 1, 0],
        "ALDf": [0, 1, 2, 3, 4, 5],
        "ALDr": [0, 1, 2, 3, 4, 5],
    },
)

if (
    concs := (
        Simulator(
            mapper.build_model(
                concs=concs.iloc[-1],
                fluxes=fluxes.iloc[-1],
                initial_labels={"GAP": 0},
            )
        )
        .simulate(20)
        .get_full_concs()
    )
) is not None:
    plot.relative_label_distribution(mapper, concs, n_cols=3)

<div style="color: #ffffff; background-color: #04AA6D; padding: 3rem 1rem 3rem 1rem; box-sizing: border-box">
    <h2>First finish line</h2>
    With that you now know most of what you will need from a day-to-day basis about labelled models in modelbase2.
    <br />
    <br />
    Congratulations!
</div>

## ToDo

- initial labels
- external labels